In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

In [ ]:
data = {"LCDM": {"Groups": np.array([[1,0], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_fix"},
        "p082": {"Groups": np.array([[1,0], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_curvaton_p082_fix"}, 
        "p084": {"Groups": np.array([[1,0], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_curvaton_p084_fix"}}

filename = 'main_halo_coords_naive.png'

In [ ]:
data = {"LCDM": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_fix"},
        "p082": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_curvaton_p082_fix"}, 
        "p084": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [3,0], [4,0]]), "SimID": "V1_LR_curvaton_p084_fix"}}

filename = 'main_halo_coords.png'

In [ ]:
for key, sim_data in data.items():
    sim_id = sim_data["SimID"]
    snap = snapshot_obj.Snapshot(sim_id, 127)
    cops = snap.get_subhalos("CentreOfPotential")
    coords = dataset_compute.group_particles_by_subhalo(snap, "Coordinates")["Coordinates"]
    
    cops_sel = []
    coords_sel = []
    for group in sim_data["Groups"]:
        idx = snap.index_of_halo(group[0], group[1])
        cops_sel.append(cops[idx] * units.cm.to(units.Mpc))
        coords_sel.append(coords[idx] * units.cm.to(units.Mpc))
    
    data[key]["CentreOfPotential"] = cops_sel
    data[key]["Coordinates"] = coords_sel

In [ ]:
projs = [["x", "y", 0, 1],
         ["x", "z", 0, 2],
         ["y", "z", 1, 2]]

fig, axes = plt.subplots(nrows=len(data), ncols=len(projs))
plt.subplots_adjust(wspace=0.8)
plt.subplots_adjust(hspace=0.8)


for ax_col in axes.T:        
    ax_col[0].set_xlim(7,12)
    ax_col[0].set_ylim(13,22)
    ax_col[1].set_xlim(7,12)
    ax_col[1].set_ylim(83,86)
    ax_col[2].set_xlim(13,22)
    ax_col[2].set_ylim(83,86)
    

col = [['pink', 'red'], ['lightblue', 'blue'], ['lightgreen', 'green'], ['yellow', 'orange'], ['violet', 'purple'], ['gray', 'black']]

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):

    axes[0, i].set_title(key)
    
    # Iterate through projections (rows):
    for j, ax_set in enumerate(projs):
        axes[j,i].set_xlabel("${}$".format(ax_set[0]))
        axes[j,i].set_ylabel("${}$".format(ax_set[1]))
        
        n_groups = len(sim_data['Groups'])
        legends = [None] * n_groups
        # Iterate through halos:
        for idx, k in enumerate(range(n_groups)):
            cop_x = sim_data["CentreOfPotential"][k][ax_set[2]]
            cop_y = sim_data["CentreOfPotential"][k][ax_set[3]]
            part_x = sim_data["Coordinates"][k][:, ax_set[2]]
            part_y = sim_data["Coordinates"][k][:, ax_set[3]]
            axes[j,i].scatter(part_x, part_y, c=col[k][0], s=0.1)
            legends[idx] = axes[j,i].scatter(cop_x, cop_y, c=col[k][1], s=0.1)
   
labels = ["{}, {}".format(grp[0],grp[1]) for grp in data["LCDM"]["Groups"]]
plt.legend(legends, labels, 
           bbox_to_anchor=(1.1, 2.0),
           title="GN, SGN")
# Adjust the scaling factor to fit your legend text completely outside the plot
# (smaller value results in more space being made for the legend)
plt.subplots_adjust(right=0.78)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)
plt.savefig(filename, dpi=300)